### Upload initial dataset to huggingface

In [1]:
from huggingface_hub import HfApi

api = HfApi()

# api.upload_folder(
#     folder_path="semeval/experiments/kosenko/language_bind/train_results/kosenko_exp_1",
#     repo_id="dim/SemEvalParticipants_models",
#     path_in_repo="kosenko_exp_1",
#     repo_type="model",
#     # ignore_patterns=["*.mp4"],
#     token=open("/code/hf_token").read(),
# )
# api.upload_file(
#     path_or_fileobj="SemEval-2024_Task3/training_data/all_videos_and_audios.tar.gz",
#     path_in_repo="training_data/all_videos_and_audios.tar.gz",
#     repo_id="dim/SemEvalSubtask2",
#     repo_type="dataset",
#     token=open("./hf_token").read(),
# )

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Explore Dataset

In [3]:
from datasets import load_dataset
from torchvision.io import read_video
import json
import torch
import os

In [4]:
dataset = json.loads(
    open("./SemEval-2024_Task3/training_data/Subtask_2_train.json").read()
)


# dataset = json.loads(open("SemEval-2024_Task3/trial_data/Subtask_2_trial.json").read())

In [5]:
dataset[0]

{'conversation_ID': 1,
 'conversation': [{'utterance_ID': 1,
   'text': 'Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .',
   'speaker': 'Chandler',
   'emotion': 'neutral',
   'video_name': 'dia1utt1.mp4'},
  {'utterance_ID': 2,
   'text': 'Oh , yeah . Had that dream .',
   'speaker': 'All',
   'emotion': 'neutral',
   'video_name': 'dia1utt2.mp4'},
  {'utterance_ID': 3,
   'text': 'Then I look down , and I realize there is a phone ... there .',
   'speaker': 'Chandler',
   'emotion': 'surprise',
   'video_name': 'dia1utt3.mp4'},
  {'utterance_ID': 4,
   'text': 'Instead of ... ?',
   'speaker': 'Joey',
   'emotion': 'surprise',
   'video_name': 'dia1utt4.mp4'},
  {'utterance_ID': 5,
   'text': 'That is right .',
   'speaker': 'Chandler',
   'emotion': 'anger',
   'video_name': 'dia1utt5.mp4'},
  {'utterance_ID': 6,
   'text': 'Never had that dream .',
   'speaker': 'Joey',
   'emotion': 'neutral',
   'video_nam

In [6]:
dataset[1]

{'conversation_ID': 2,
 'conversation': [{'utterance_ID': 1,
   'text': 'I do not want to be single , okay ? I just ... I just ... I just wanna be married again !',
   'speaker': 'Ross',
   'emotion': 'sadness',
   'video_name': 'dia2utt1.mp4'},
  {'utterance_ID': 2,
   'text': 'And I just want a million dollars !',
   'speaker': 'Chandler',
   'emotion': 'neutral',
   'video_name': 'dia2utt2.mp4'},
  {'utterance_ID': 3,
   'text': 'Rachel ? !',
   'speaker': 'Monica',
   'emotion': 'surprise',
   'video_name': 'dia2utt3.mp4'}],
 'emotion-cause_pairs': [['1_sadness', '1'], ['3_surprise', '3']]}

In [7]:
dataset[2]

{'conversation_ID': 3,
 'conversation': [{'utterance_ID': 1,
   'text': 'Oh my God !',
   'speaker': 'Monica',
   'emotion': 'surprise',
   'video_name': 'dia3utt1.mp4'},
  {'utterance_ID': 2,
   'text': 'I know , I know , I am such an idiot .',
   'speaker': 'Paul',
   'emotion': 'neutral',
   'video_name': 'dia3utt2.mp4'},
  {'utterance_ID': 3,
   'text': 'I guess I should have caught on when she started going to the dentist four and five times a week . I mean , how clean can teeth get ?',
   'speaker': 'Paul',
   'emotion': 'sadness',
   'video_name': 'dia3utt3.mp4'},
  {'utterance_ID': 4,
   'text': 'My brother going through that right now , he is such a mess . How did you get through it ?',
   'speaker': 'Monica',
   'emotion': 'neutral',
   'video_name': 'dia3utt4.mp4'},
  {'utterance_ID': 5,
   'text': 'Well , you might try accidentally breaking something valuable of hers , say her ...',
   'speaker': 'Paul',
   'emotion': 'neutral',
   'video_name': 'dia3utt5.mp4'},
  {'utteran

In [5]:
dataset[3]

{'conversation_ID': 4,
 'conversation': [{'utterance_ID': 1,
   'text': 'Barry , I am sorry ...',
   'speaker': 'Rachel',
   'emotion': 'sadness',
   'video_name': 'dia4utt1.mp4'},
  {'utterance_ID': 2,
   'text': 'I am so sorry ...',
   'speaker': 'Rachel',
   'emotion': 'sadness',
   'video_name': 'dia4utt2.mp4'},
  {'utterance_ID': 3,
   'text': 'I know you probably think that this is all about what I said the other day about you making love with your socks on , but it is not ... it is not , it is about me , and I ju ...',
   'speaker': 'Rachel',
   'emotion': 'sadness',
   'video_name': 'dia4utt3.mp4'}],
 'emotion-cause_pairs': []}

In [9]:
dataset[10]

{'conversation_ID': 11,
 'conversation': [{'utterance_ID': 1,
   'text': 'I mean , why should I let them meet him ? I mean , I bring a guy home , and within five minutes they are all over him . I mean , they are like ... coyotes , picking off the weak members of the herd .',
   'speaker': 'Monica',
   'emotion': 'disgust',
   'video_name': 'dia11utt1.mp4'},
  {'utterance_ID': 2,
   'text': 'Listen . As someone who seen more than her fair share of bad beef , I will tell you : that is not such a terrible thing .',
   'speaker': 'Paula',
   'emotion': 'neutral',
   'video_name': 'dia11utt2.mp4'},
  {'utterance_ID': 3,
   'text': 'Come on , they are your friends , they are just looking out after you .',
   'speaker': 'Paula',
   'emotion': 'neutral',
   'video_name': 'dia11utt3.mp4'},
  {'utterance_ID': 4,
   'text': 'I know . I just wish that once , I would bring a guy home that they actually liked .',
   'speaker': 'Monica',
   'emotion': 'sadness',
   'video_name': 'dia11utt4.mp4'},
  {

In [10]:
dataset[100]

{'conversation_ID': 130,
 'conversation': [{'utterance_ID': 1,
   'text': 'Four letters : " Circle or hoop " .',
   'speaker': 'Ross',
   'emotion': 'neutral',
   'video_name': 'dia130utt1.mp4'},
  {'utterance_ID': 2,
   'text': 'Ring dammit , ring !',
   'speaker': 'Chandler',
   'emotion': 'anger',
   'video_name': 'dia130utt2.mp4'},
  {'utterance_ID': 3,
   'text': 'Thanks .',
   'speaker': 'Ross',
   'emotion': 'neutral',
   'video_name': 'dia130utt3.mp4'},
  {'utterance_ID': 4,
   'text': 'Hey , you know our phone not working ?',
   'speaker': 'Joey',
   'emotion': 'surprise',
   'video_name': 'dia130utt4.mp4'},
  {'utterance_ID': 5,
   'text': 'What ? !',
   'speaker': 'Chandler',
   'emotion': 'surprise',
   'video_name': 'dia130utt5.mp4'},
  {'utterance_ID': 6,
   'text': 'I tried to call you from the coffee shop , and there was no answer .',
   'speaker': 'Joey',
   'emotion': 'neutral',
   'video_name': 'dia130utt6.mp4'},
  {'utterance_ID': 7,
   'text': 'I turned it off . Mo

In [6]:
len(dataset)

1374

In [7]:
import datasets

all_data = datasets.Dataset.from_list(dataset)
all_data = all_data.train_test_split(
    test_size=0.08,
    seed=42,
)

In [8]:
all_data

DatasetDict({
    train: Dataset({
        features: ['conversation_ID', 'conversation', 'emotion-cause_pairs'],
        num_rows: 1264
    })
    test: Dataset({
        features: ['conversation_ID', 'conversation', 'emotion-cause_pairs'],
        num_rows: 110
    })
})

In [9]:
all_data.push_to_hub('dim/semeval_subtask2_conversations')

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.14s/it]


In [11]:
all_data['train'][0]

{'conversation_ID': 187,
 'conversation': [{'emotion': 'sadness',
   'speaker': 'Monica',
   'text': 'Mr . Heckles .',
   'utterance_ID': 1,
   'video_name': 'dia187utt1.mp4'},
  {'emotion': 'neutral',
   'speaker': 'Rachel',
   'text': 'How did this happen ?',
   'utterance_ID': 2,
   'video_name': 'dia187utt2.mp4'},
  {'emotion': 'neutral',
   'speaker': 'Mr. Treeger',
   'text': 'He musta been sweeping . They found a broom in his hand .',
   'utterance_ID': 3,
   'video_name': 'dia187utt3.mp4'},
  {'emotion': 'sadness',
   'speaker': 'Monica',
   'text': 'That is terrible .',
   'utterance_ID': 4,
   'video_name': 'dia187utt4.mp4'},
  {'emotion': 'neutral',
   'speaker': 'Mr. Treeger',
   'text': 'I know . I was sweeping yesterday . It coulda been me .',
   'utterance_ID': 5,
   'video_name': 'dia187utt5.mp4'},
  {'emotion': 'neutral',
   'speaker': 'Ross',
   'text': 'Sure , you coulda . You never know .',
   'utterance_ID': 6,
   'video_name': 'dia187utt6.mp4'},
  {'emotion': 'neu